<a href="https://colab.research.google.com/github/ma2bara/GeisterWorkshop/blob/master/GeisterWorkshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

これは、ボードAIワークショップ第1回　ガイスター用のサンプルプログラムです。

In [5]:
# coding:utf-8
"""
Geister program for Board game AI Workshop #1
written by matsubara@morikatron.co.jp
2019/5/20

動作環境はPython 3.5以降。特別なライブラリは不要。
"""

from enum import Enum
from typing import List, Tuple, Union
import random
import re

# ゲームの基本的な枠組みや表現に関する各種の定数を宣言
BOARD_WIDTH = 6  # ボードの幅
BOARD_HEIGHT = 6  # 高さ
MAX_PIECES = 8  # プレイヤーの持ち駒の数
ME = 0  # pieces[]リストの先頭はAIプレイヤー（自分=Me）の持ち駒とする
OP = 1  # pieces[]リストの2番目は敵プレイヤー（相手=Opponent）の持ち駒とする
NO_PLAYER = -1  # MEでもOPでもない場合に使う値
LOC_CAPTURED = 99  # x,yにLOC_CAPTUREDが入っていれば、そのコマは相手に取られたコマとする
LOC_ESCAPED_W = -1  # xにLOC_ESCAPED_Wが入っていれば、そのコマ（青のみだが）は敵陣地から抜け出た（勝った）コマとする
LOC_ESCAPED_E = BOARD_WIDTH  # xにLOC_ESCAPED_Eが入っていれば、そのコマ（青のみだが）は敵陣地から抜け出た（勝った）コマとする
COL_R = -1.0  # 赤
COL_B = 1.0  # 青
COL_U = 0.0  # 不明

# 表示制御に関する定数
ZENKAKU = False   # Trueにすると全角で表示。全角で等幅フォントが正しく表示できる環境なら全角のほうが快適な感じ。
                  # Colabでは、ブラウザの固定幅フォントを正しく指定すれば半角でズレずに表示できることを確認。
if ZENKAKU:
    CHAR_RED = 'ｒ'  # 'ⓡ'
    CHAR_BLUE = 'ｂ'  # 'ⓑ'
    CHAR_UNDEFINED = '？'  # '🅤'
    CHAR_SPACE = '・'
else:
    CHAR_RED = 'r'
    CHAR_BLUE = 'b'
    CHAR_UNDEFINED = '?'
    CHAR_SPACE = '-'


# ゲームの進行を保持・制御する列挙型を宣言
class GameState(Enum):
    enter_f_or_s = 0  # ゲームの開始待ち（どちらが先手か入力してもらう）
    enter_opponent_move = 1  # 相手の打ち手（x,y,direction）の入力待ち
    enter_color_of_captured_piece = 2  # AIがとったコマの色の入力待ち
    next_is_AI_move = 3  # 次はAIの番です
    won = 4  # 私の勝ち
    lost = 5  # 私の負け


# クラスを定義
class Piece:
    """1つのコマに関する情報を記録するクラス"""

    def __init__(self, x: int, y: int, col: float):
        """初期化"""
        self.x = x  # x座標
        self.y = y  # y座標
        self.color = col  # 色
        self.e_color = 0.0  # COL_Uの推測値 Estimated value を保持。　COL_R <= e_color <= COL_B の値をとるとする。

    def get_color_string(self) -> str:
        """colorの値を文字列に変換して返す。推定値も。"""
        if self.color == COL_R:
            return "R"
        elif self.color == COL_B:
            return "B"
        elif self.e_color == COL_R:
            return "?R"
        elif self.e_color == COL_B:
            return "?B"
        return "?" + str(self.e_color)

    def __repr__(self):
        """Pieceのインスタンスがprint()で表示できるようにするための関数"""
        return "(%d,%d,%s)" % (self.x, self.y, self.get_color_string())


class Player:
    """一人のプレイヤーの状態をすべて記録するクラス"""

    def __init__(self, which_player: int = ME, pieces: List[Piece] = None):
        self.which_player = which_player  # 自分か相手かを保持
        self.pieces = pieces  # 初期化時に８個のコマが入ったリストを受け取る
        self.n_alive_pieces = len(self.pieces)  # 生きているコマの数
        self.n_escaped = 0  # 敵陣から抜けたコマの数
        self.n_alive_red = 0  # 生きている赤コマの数
        self.n_alive_blue = 0  # 生きている青コマの数
        self.n_captured_pieces = 0  # 捕獲されたコマの数
        self.n_captured_red = 0  # 捕獲された赤コマの数
        self.n_captured_blue = 0  # 捕獲された青コマの数

    def analyse(self):
        """コマの生死状態をカウントする"""
        self.n_alive_pieces = 0  # 生きているコマの数
        self.n_escaped = 0  # 敵陣から抜けたコマの数
        self.n_alive_red = 0  # 生きている赤コマの数
        self.n_alive_blue = 0  # 生きている青コマの数
        self.n_captured_pieces = 0  # 捕獲されたコマの数
        self.n_captured_red = 0  # 捕獲された赤コマの数
        self.n_captured_blue = 0  # 捕獲された青コマの数
        for piece in self.pieces:
            self.n_alive_pieces = self.n_alive_pieces + 1
            if piece.x in {LOC_ESCAPED_W, LOC_ESCAPED_E}:  # 脱出カウント
                self.n_escaped = self.n_escaped + 1
            if piece.x == LOC_CAPTURED:
                self.n_captured_pieces = self.n_captured_pieces + 1
                if piece.color == COL_B:
                    self.n_captured_blue = self.n_captured_blue + 1
                elif piece.color == COL_R:
                    self.n_captured_red = self.n_captured_red + 1
            else:
                self.n_alive_pieces = self.n_alive_pieces + 1
                if piece.color == COL_B:
                    self.n_alive_blue = self.n_alive_blue + 1
                elif piece.color == COL_R:
                    self.n_alive_red = self.n_alive_red + 1


class Move:
    """一つの手を表現するクラス"""
    news = ['n', 'e', 'w', 's']

    def __init__(self, which_player=ME, piece_ix=-1, piece_x=0, piece_y=0, direction='n'):
        self.which_player = which_player  # プレイヤー番号
        self.piece_ix = piece_ix  # コマの番号
        self.piece_x = piece_x  # コマの番号
        self.piece_y = piece_y  # コマの番号
        self.direction = direction  # コマの移動方向
        self.x_after_move = 0  # コマを動かした後のX座標
        self.y_after_move = 0  # コマを動かした後のY座標
        if self.piece_ix < 0:
            # piece_xとpiece_yから対象pieceを探し出す
            which_player, found_piece = find_piece_from_xy(piece_x, piece_y)
            if found_piece is None:
                print("error! piece (" + str(piece_x) + ", " + str(piece_y) + ") was not found.")
            else:
                global CurPlayers
                self.piece_ix = CurPlayers[which_player].pieces.index(found_piece)
        else:
            # piece_ixからxとyを出しておく
            self.piece_x = CurPlayers[self.which_player].pieces[self.piece_ix].x
            self.piece_y = CurPlayers[self.which_player].pieces[self.piece_ix].y
        self.calc_moved_loc()  # コマを動かした後の座標を計算する

    @classmethod
    def rand(cls, which_player=ME):
        """完全にランダムな打ち手（妥当性は考慮せず）のインスタンスを生成して返す"""
        piece_ix = random.randrange(MAX_PIECES)
        direction = Move.news[random.randrange(4)]
        return cls(which_player=which_player, piece_ix=piece_ix, direction=direction)

    def calc_moved_loc(self):
        """Moveを実行した結果、コマがどこへ移動するかを計算しておく"""
        x = self.piece_x
        y = self.piece_y
        if self.direction == 'n':
            y = y - 1
        if self.direction == 'e':
            x = x + 1
        if self.direction == 'w':
            x = x - 1
        if self.direction == 's':
            y = y + 1
        self.x_after_move = x  # コマを動かした後のX座標
        self.y_after_move = y  # コマを動かした後のY座標

    def __repr__(self):
        """Moveのインスタンスをprint()で表示するための関数"""
        global CurPlayers
        return "(%d,%d,%s)" % (CurPlayers[self.which_player].pieces[self.piece_ix].x,
                               CurPlayers[self.which_player].pieces[self.piece_ix].y,
                               self.direction)

    def reverse_repr(self):
        """敵の側から見た打ち手を返す（相手に伝える時に逆から見た時の x,y,direction を伝えると楽に進行できそうだから）"""
        x = (BOARD_WIDTH - 1) - CurPlayers[self.which_player].pieces[self.piece_ix].x
        y = (BOARD_HEIGHT - 1) - CurPlayers[self.which_player].pieces[self.piece_ix].y
        if self.direction == 'n':
            d = 's'
        elif self.direction == 'e':
            d = 'w'
        elif self.direction == 'w':
            d = 'e'
        else:  # 's'
            d = 'n'
        return "(%d,%d,%s)" % (x, y, d)


# ここでグローバル変数を定義
CurGameState: GameState = GameState.enter_f_or_s  # 現在のゲームの状態を保持する変数
CurPlayers = []  # Player情報を保持するリスト。要素の数は２で、Player[ME]はAI（自分）、Player[OP]は敵とする
Which_player_is_first: int = ME  # 先手のプレイヤーを記録しておく
Last_captured_piece: Piece  # 最後に獲得された相手のコマ
LastMove = None  # 最後に動かした手を記憶しておいて、bordに矢印で表示するとわかりやすいかも


# 捕獲されたコマ４個目までのRB表示
def get_captured_piece_strings(player):
    ix = 0
    result = ""
    for piece in player.pieces:
        if piece.x == LOC_CAPTURED:
            if piece.color == COL_R:
                result = result + CHAR_RED
            elif piece.color == COL_B:
                result = result + CHAR_BLUE
            else:
                result = result + CHAR_UNDEFINED
            ix = ix + 1
    for i in range(8 - ix):
        if ZENKAKU:
            result = result + '　'
        else:
            result = result + ' '
    return result[0:4], result[4:8]


def show_board() -> None:
    """現在のボード状態を表示する"""
    """ 出力フォーマットは以下。全角でもずれなければZENKAKU = Trueで。
    　　　　　　　　　 Ｎ
    　　　　　　　０１２３４５　
    ⓡⓡⓡⓡ　０　・？？？？・　５
    ⓑⓑⓑⓑ　１　・？？？？・　４
    　　　　Ｗ２　・・・・・・　３Ｅ
    　　　　　３　・・・・・・　２
    　　　　　４　・ⓡⓡⓡⓡ・　１　ⓡⓡⓡⓡ
    　　　　　５　・？？？？・　０　ⓑⓑⓑⓑ
    　　　　　　　５４３２１０
    　　　　　　　　　 Ｓ
    """
    # print(CurPlayers[OP].pieces)  # show pieces for debug
    # print(CurPlayers[ME].pieces)  # show pieces for debug
    # 盤面の中を作成
    if ZENKAKU:
        board = ["　・・・・・・　"] * BOARD_HEIGHT
    else:
        board = [" ------ "] * BOARD_HEIGHT
    for p in CurPlayers:
        for piece in p.pieces:
            if piece.x != LOC_CAPTURED:
                if piece.color == COL_R:
                    board[piece.y] = board[piece.y][:piece.x + 2 - 1] + CHAR_RED + board[piece.y][piece.x + 2:]
                elif piece.color == COL_B:
                    board[piece.y] = board[piece.y][:piece.x + 2 - 1] + CHAR_BLUE + board[piece.y][piece.x + 2:]
                else:
                    board[piece.y] = board[piece.y][:piece.x + 2 - 1] + CHAR_UNDEFINED + board[piece.y][piece.x + 2:]
    my_captured_string_1, my_captured_string_2 = get_captured_piece_strings(CurPlayers[ME])
    op_captured_string_1, op_captured_string_2 = get_captured_piece_strings(CurPlayers[OP])
    # 表示
    if ZENKAKU:
        print("　　　　　　　　　 ｎ")
        print("　　　　　　　０１２３４５")
        print(my_captured_string_1 + "　０" + board[0] + "５")
        print(my_captured_string_2 + "　１" + board[1] + "４")
        print("　　　ｗ　２" + board[2] + "３　ｅ")
        print("　　　　　３" + board[3] + "２")
        print("　　　　　４" + board[4] + "１　" + op_captured_string_1)
        print("　　　　　５" + board[5] + "０　" + op_captured_string_2)
        print("　　　　　　　５４３２１０")
        print("　　　　　　　　　 ｓ")
    else:
        print("          n")
        print("       012345")
        print(my_captured_string_1 + " 0" + board[0] + "5")
        print(my_captured_string_2 + " 1" + board[1] + "4")
        print("   w 2" + board[2] + "3 e")
        print("     3" + board[3] + "2")
        print("     4" + board[4] + "1 " + op_captured_string_1)
        print("     5" + board[5] + "0 " + op_captured_string_2)
        print("       543210")
        print("          s")


def show_status_message() -> None:
    """現在の状況（何を待っているかなど）を表示する　"""
    global CurGameState
    if CurGameState == GameState.enter_f_or_s:  # ゲーム開始待ちなので、先手か後手かを入れてくれ
        print('enter f(I am first) or s(I am second)')
    elif CurGameState == GameState.enter_opponent_move:  # 相手の手番なので、相手の手を入れてくれ
        print('enter opponent move x,y,n/e/w/s (e.g. 1,2,s)')
    elif CurGameState == GameState.next_is_AI_move:  # 次はAIの番ですよ
        print('AI Thinking ...')
    elif CurGameState == GameState.enter_color_of_captured_piece:  # AIがとったコマの色の入力待ち
        print('enter r or b (color of captured piece)')
    elif CurGameState == GameState.won:  # 勝った表示
        print('I won!')
    elif CurGameState == GameState.lost:  # 負けた表示
        print('I lost.')


def show_help() -> None:
    """入力できるコマンドなどの説明を表示する　"""
    print('----------')
    print('h, help, ?     : show help')
    print('q, quit        : quit program')
    print('e, end, finish : finish game, and start new game')
    print('----------')


def reset_game() -> None:
    """ゲームの状態をすべてリセットして、ゲームを開始できる状態にする"""
    global CurGameState, CurPlayers, LastMove
    CurGameState = GameState.enter_f_or_s  # 現在のゲームの状態を保持する変数
    LastMove = None  # 最後に動かした手
    """ ゲーム開始時のコマの配置場所を決めます
     012345
    0 0123 5  　　←　こちらが敵側とします
    1 4567 4
    2      3
    3      2
    4 0123 1　　　←　こちらが自分側とします
    5 4567 0
     543210
    """
    # 自分（AI）のコマ情報を保持するplayerを作ります
    # 前４個を赤に、後ろ４個を青にしています。これは各自のAIで好きな配置を選んでください（何らかのアルゴリズムで配置する仕組みを作っても良いですね）
    me = Player(which_player=ME, pieces=[
        Piece(1, 4, COL_R), Piece(2, 4, COL_R), Piece(3, 4, COL_R), Piece(4, 4, COL_R),
        Piece(1, 5, COL_B), Piece(2, 5, COL_B), Piece(3, 5, COL_B), Piece(4, 5, COL_B)
    ])
    # 相手（敵）のコマ情報を保持するplayerを作ります
    # 敵のコマは色が不明なのでCOL_Uで全部並べます
    op = Player(which_player=OP, pieces=[
        Piece(1, 0, COL_U), Piece(2, 0, COL_U), Piece(3, 0, COL_U), Piece(4, 0, COL_U),
        Piece(1, 1, COL_U), Piece(2, 1, COL_U), Piece(3, 1, COL_U), Piece(4, 1, COL_U)
    ])
    # playerリストに保存します
    CurPlayers = [me, op]


def is_game_over() -> bool:
    """盤面解析して勝利条件が確定しているか確かめ、確定していればTrueを返す"""
    # すでに勝ち負けが決まっていればTrueを返すだけでよい
    global CurGameState, CurPlayers
    if CurGameState in {GameState.won, GameState.lost}:
        return True
    # いろいろカウント
    CurPlayers[ME].analyse()
    CurPlayers[OP].analyse()
    # 各勝利条件を調べていく
    # 敵陣を抜けたコマがいる　＝　勝ち
    if CurPlayers[ME].n_escaped > 0:
        CurGameState = GameState.won
        return True
    # 自陣から抜けられたコマがいる　＝　負け
    if CurPlayers[OP].n_escaped > 0:
        CurGameState = GameState.lost
        return True
    # 敵の赤を４個取ってしまった判定　＝　負け
    if CurPlayers[OP].n_captured_red >= 4:
        CurGameState = GameState.lost
        return True
    # 敵の青を４個取ってしまった判定　＝　勝ち
    if CurPlayers[OP].n_captured_blue >= 4:
        CurGameState = GameState.won
        return True
    # 自分の赤を４個取られてしまった判定　＝　勝ち
    if CurPlayers[ME].n_captured_red >= 4:
        CurGameState = GameState.won
        return True
    # 自分の青を４個取られてしまった判定　＝　負け
    if CurPlayers[ME].n_captured_blue >= 4:
        CurGameState = GameState.lost
        return True
    # 上記状態以外であれば、ゲーム終了条件は成立していないのでFalseを返す
    return False


def find_piece_from_xy(x: int, y: int) -> Union[Tuple[int, None], Tuple[int, Piece]]:
    """XYで指定された座標に存在するコマを探して、誰のどのコマかを返す（または何もない=NO_PLAYERを返す）"""
    global CurPlayers
    for p in CurPlayers:
        for piece in p.pieces:
            if piece.x == x and piece.y == y:
                return p.which_player, piece
    return NO_PLAYER, None


def is_correct_move(move: Move) -> bool:
    """手が適正かどうかを判定します"""
    global CurPlayers
    if move.piece_ix < 0 or move.piece_ix >= MAX_PIECES:
        return False
    target_piece = CurPlayers[move.which_player].pieces[move.piece_ix]
    # 指定したコマがすでに脱出していたらFalseを返す
    if target_piece.x in {LOC_ESCAPED_W, LOC_ESCAPED_E}:
        return False
    # 指定したコマがすでに捕獲されていたらFalseを返す
    if target_piece.x == LOC_CAPTURED:
        return False
    # 移動が「青コマの脱出」であればTrueを返す
    if target_piece.color != COL_R:  # 敵のUnknownのコマでも脱出行動は適正と判断したいので !COL_R で比較
        if (move.which_player == ME and target_piece.y == 0) or \
                (move.which_player == OP and target_piece.y == (BOARD_HEIGHT - 1)):  # 自分なら上、敵なら下の行で
            if target_piece.x == 0:  # 左から
                if move.direction == 'w':  # 西（＝左）へ抜けようとしていたら
                    return True  # 適正
            elif target_piece.x == (BOARD_WIDTH - 1):  # 右から
                if move.direction == 'e':  # 東（＝右）へ抜けようとしていたら
                    return True  # 適正
    # 移動先が盤面からはみ出していればFalseを返す
    if move.x_after_move < 0 or move.x_after_move >= BOARD_WIDTH:
        return False
    if move.y_after_move < 0 or move.y_after_move >= BOARD_HEIGHT:
        return False
    # 移動先に自分のコマがいたらFalseを返す
    which_player, _ = find_piece_from_xy(move.x_after_move, move.y_after_move)
    if move.which_player == which_player:  # 移動先に自分のコマがいるなら、の条件安定
        return False
    # 上記以外の条件ならTrue（適正な打ち手）と判断してTrueを返す
    return True


def execute_move(move: Move) -> Union[Piece, None]:
    """打ち手を実行して盤面をアップデートする。与えられた手Moveは適正なものとする（事前にis_correct_moveでチェック済みであるとする）。とったコマを返す"""
    global CurPlayers, LastMove
    LastMove = move
    target_piece = CurPlayers[move.which_player].pieces[move.piece_ix]
    # 移動先にコマがあれば、それを発見しておく
    which_player, captured_piece = find_piece_from_xy(move.x_after_move, move.y_after_move)
    # 次にコマを移動する
    target_piece.x = move.x_after_move
    target_piece.y = move.y_after_move
    # 移動先でコマが見つかっていた場合は、それを獲得状態に変更する
    if which_player != NO_PLAYER:
        # 移動先にコマがあるので、それを獲得する
        captured_piece.x = LOC_CAPTURED
        captured_piece.y = LOC_CAPTURED
        return captured_piece
    return None  # 相手のコマはとっていません


def think() -> GameState:
    """次の手を考え、打ち、状況を判定して、次のゲームステータスを返す"""
    global CurGameState
    # まず最初にゲームが終了していないことを確認します（念のため）
    if is_game_over():
        return CurGameState
    # 次の手を考えます。このサンプルではランダムな手を選択します。
    while True:
        move = Move.rand()  # ランダムな手を入手
        if is_correct_move(move):  # 打てる手ならそれでヨシとする
            break

    # この段階で、打つ手が move に決定した、とします。

    # コンソールに「このコマをこう動かします」と表示します。
    print("--------------------------")
    print("AI move is " + str(move) + " ⇄ " + move.reverse_repr())
    print("--------------------------")
    # AIの考えた手を打ちます
    captured_piece = execute_move(move)
    if captured_piece is not None:
        # 敵のコマをとった場合、そのコマの色を入力してもらいます
        global Last_captured_piece
        Last_captured_piece = captured_piece
        return GameState.enter_color_of_captured_piece  # 色の入力待ちに遷移
    # ゲームの終了条件を判定します
    if is_game_over():
        return CurGameState
    # GameStatusをアップデートします（次の打ち手待ちになるように）
    return GameState.enter_opponent_move  # AIが考えた後は敵の打ち手を待つ状態に遷移


def opponent_move(move: Move) -> CurGameState:
    """敵の手を実行、状況を判定して、次のゲームステータスを返す"""
    global CurGameState
    # まず最初にゲームが終了していないことを確認します（念のため）
    if is_game_over():
        return CurGameState
    # 敵の考えた手を打ちます
    captured_piece = execute_move(move)
    if captured_piece is not None:
        # AIのコマをとった場合
        global Last_captured_piece
        Last_captured_piece = captured_piece
    # ゲームの終了条件を判定します
    if is_game_over():
        return CurGameState
    # GameStatusをアップデートします（次の打ち手待ちになるように）
    return GameState.next_is_AI_move  # 敵が考えた後はAIの打ち手を待つ状態に遷移


def process_command(cmd: str) -> bool:
    """入力を処理する。処理できたらTrueを、意味不明の場合はFalseを返す"""
    global CurGameState, Which_player_is_first
    if CurGameState == GameState.enter_f_or_s:  # ゲーム開始待ちなので、先手か後手かを入れてくれ
        # print('enter f(I am first) or s(I am second)')
        if cmd == 'f':
            # 先手を選択された　＝　最初のAIの手を考えて実行
            Which_player_is_first = ME
            CurGameState = think()
            return True
        if cmd == 's':
            # 後手を選択された = 敵のコマの入力待ちになる
            Which_player_is_first = OP
            CurGameState = GameState.enter_opponent_move
            return True
        print('f または s を入力してください。 fはAIが先手, sはAIが後手の意味です。')
        return False
    elif CurGameState == GameState.enter_opponent_move:  # 相手の手番なので、相手の手を入れてくれ
        # print('enter opponent move x,y,n/e/w/s (e.g. 1,2,s)')
        # commands = cmd.split(",")
        commands = re.split(r'\s|"|,|\.', cmd)
        if len(commands) != 3 or \
                not commands[0].isdecimal or \
                len(commands[0]) < 1 or \
                not commands[1].isdecimal or \
                len(commands[1]) < 1 or \
                commands[2] not in {'n', 'e', 's', 'w'}:
            print("相手の指し手を 0,1,s のように x,y,方角 の形で入力してください")
            return False
        move = Move(which_player=OP,
                    piece_ix=-1,
                    piece_x=int(commands[0]),
                    piece_y=int(commands[1]),
                    direction=commands[2])
        if is_correct_move(move):
            CurGameState = opponent_move(move)
        else:
            print("指定された手(" + cmd + ")は打てません。コマが見つからないか、不正な移動です。")
        return True
    elif CurGameState == GameState.enter_color_of_captured_piece:  # AIがとったコマの色の入力待ち
        # print('enter r or b (color of captured piece)')
        global Last_captured_piece
        if cmd in {'r', 'red'}:
            Last_captured_piece.color = COL_R
        elif cmd in {'b', 'blue'}:
            Last_captured_piece.color = COL_B
        else:
            return False
        if not is_game_over():
            CurGameState = GameState.enter_opponent_move
        return True
    elif CurGameState == GameState.won:  # 勝った表示
        return True
    elif CurGameState == GameState.lost:  # 負けた表示
        return True
    return False


def main() -> None:
    global CurGameState
    reset_game()
    while True:
        # 現在のボード状態を表示する
        show_board()
        # 現在の状況に応じた入力を催促する
        show_status_message()
        if CurGameState == GameState.next_is_AI_move:  # 次はAIの番ですよ
            # AIに考えて打ってもらう
            CurGameState = think()
        else:
            # 何らかのコマンドを入れてもらう
            cmd = input('>> ')
            cmd = cmd.lower()
            # 終了コマンドの検出と処理
            if cmd in {'quit', 'q'}:
                break
            # ヘルプコマンドの検出と処理
            if cmd in {'help', 'h', '?'}:
                show_help()
                continue
            # endコマンドの検出と処理
            if cmd in {'e', 'end', 'finish', 'restart', 'new'}:
                reset_game()
                continue
            if process_command(cmd):
                continue


if __name__ == '__main__':
    random.seed()  # 乱数の初期化
    main()


          n
       012345
     0 -????- 5
     1 -????- 4
   w 2 ------ 3 e
     3 ------ 2
     4 -rrrr- 1     
     5 -bbbb- 0     
       543210
          s
enter f(I am first) or s(I am second)
>> f
--------------------------
AI move is (1,5,w) ⇄ (4,0,e)
--------------------------
          n
       012345
     0 -????- 5
     1 -????- 4
   w 2 ------ 3 e
     3 ------ 2
     4 -rrrr- 1     
     5 b-bbb- 0     
       543210
          s
enter opponent move x,y,n/e/w/s (e.g. 1,2,s)
>> 1 1 s
          n
       012345
     0 -????- 5
     1 --???- 4
   w 2 -?---- 3 e
     3 ------ 2
     4 -rrrr- 1     
     5 b-bbb- 0     
       543210
          s
AI Thinking ...
--------------------------
AI move is (2,4,n) ⇄ (3,1,s)
--------------------------
          n
       012345
     0 -????- 5
     1 --???- 4
   w 2 -?---- 3 e
     3 --r--- 2
     4 -r-rr- 1     
     5 b-bbb- 0     
       543210
          s
enter opponent move x,y,n/e/w/s (e.g. 1,2,s)
>> 1 2 s
          n
       012345
 